# Hands on : Perturbed initial conditions

We have been observing that the chaotic nature of the system made impossible to accurately predict future extreme events as we have incomplete information about initial state. 
We now propose to implement a Large Ensemble (LE) of simulations via the Monte Carlo algorithm to better characterise our uncertainty in our predictions.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from l96 import *
from tqdm import tqdm
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
from utils import  *
from sklearn.decomposition import PCA
np.random.seed(1)

In [ ]:
# Define colorblind-friendly colors
color_sim = '#56B4E9'   # Light blue
color_threshold = '#D55E00'  # Red

In [ ]:
initX, initY = np.load('./data/initX.npy'), np.load('./data/initY.npy')
l96_ref = L96TwoLevel(X_init=initX, Y_init=initY, save_dt=0.001, noYhist=False)
l96_ref.iterate(30)
l96_ref.erase_history()
l96_ref.iterate(0.8)

In [ ]:
l96_ref.erase_history()
l96_ref.iterate(15)
h = l96_ref.history
X = h.X.values
idx = np.arange(0, X.shape[0], 10)
X_sampled = X[idx, :]
# observations = X_sampled + np.random.normal(0, 0.3,  X_sampled.shape)

Sigma = generate_positive_definite_matrix(k=36)
E = np.random.multivariate_normal(np.repeat(0, l96_ref.K), 0.005*Sigma)
observations = X_sampled + E

threshold = np.percentile(observations, 99.9)

X_init = observations[-1, :]
Sigma = np.cov(observations.T)

In [ ]:
# Plot the covariance of the observation
# Is there correlated variables?

## How to properly choose the prior distribution?

Here we assume that we are given the background noise covariance $\Sigma$ and that the mean is $X_init$ (the last observation). 
* What would be the Maximum Entropy related to this constraints?
* You can check [Maximum Entropy Distributions](https://en.wikipedia.org/wiki/Maximum_entropy_probability_distribution)

* Generate a large ensemble of simulation by sampling $n_{\text{members}}$ over this distribution.

In [ ]:
n_members = ???
B = ???

X_perturbed = X_init + B

# Generate samples with mean X_init and random noise following the Maximum entropy prior
# Scale the covariance by a factor gamma as we wish to have a small uncertainty around X_init
# You can try different values of gamma and see how this affect the prediction

In [ ]:
# Plot the past True values, the observations and the sampled points at location X[:, 0]

* Now we can propagate our uncertainty by runing the simulation over each possible initial condition.

In [ ]:
time = 5
simulations = []
for X_init in tqdm(X_perturbed):
    l96_simulation = L96OneLevel(X_init=X_init, dt=0.001, noprog=True)

    l96_simulation.iterate(time=time)
    simulations.append(l96_simulation.history.X.data)

    l96_simulation.erase_history()

simulations = np.array(simulations)

In [ ]:
l96_ref.erase_history()
l96_ref.iterate(time = time)
X_true = l96_ref.history.X.data

We now plot the the $5-95\%$ range of our predictions together with ensemble mean and the observed value.

In [ ]:
plot_time_series(X_true[-1:, :], X_true, simulations, subsample_rate=30, alpha=5)

* Did you use enough initial sample for your Monte-Carlo simulation to converge?
* If we missed the extreme event, is it only because of the lack of initial samples?

### Convergence
* By the Central Limit Theorem, $\sqrt{n}(\bar{Y}_n - \mu_Y) \to \mathcal{N}(0, \sigma^2)$
* We can check convergence via convergence of the standard deviation

In [ ]:
# Time span on which we want to measure the convergence
time_0, time_1 = 3000, 4000

# Plot the standard deviation evolution over time

### Better sampling strategies 
* Using [scipy.stats.qmc](https://docs.scipy.org/doc/scipy/reference/stats.qmc.html), imtry other sampling strategies (e.g. Latin Hypercubes, Sobol sequences)
* Do we have faster convergence?

### Ensemble Post Processing
* In practice, it is often impracticable to run thousands of simulations
* We can get better estimation of the extremes using some post processing strategies F
    * Fitting a GEV on a few ensmeble members (see [genextreme](https://docs.scipy.org/doc/scipy/tutorial/stats/continuous_genextreme.html))
    * Using non-parametric density estimation with kernels density estimation (see [KDE](https://docs.scipy.org/doc/scipy/tutorial/stats/kernel_density_estimation.html))
* Try to implement this post-processing strategy to 

### Ensemble mean performance
* Check the ensemble mean performance compared to the first member by comparing MSEs

In [ ]:
observations = X_true
predictions = simulations.mean(axis=0)[1:, :]

In [ ]:
# Plot both the ensemble mean and first member MSE

* Is the MSE adapted if we want to compare the all predictive distribution and the observations?